In [1]:
import numpy as np

# deep learning
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.layers import Input, Dense, GRU, Dropout
from keras.layers import Bidirectional, Embedding, SpatialDropout1D, concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 100_000  # how many unique words to use (i.e num rows in embedding vector)
maxlen = 150 # max number of words in a comment to use
embed_size = 300 # how big is each word vector


nb_words = 94298
embedding_matrix = np.zeros((nb_words, embed_size))

def get_model(maxlen, max_features, embed_size, embedding_matrix,
              lr=0.0, lr_d=0.0, units=0, dr=0.0):
    """Get the tensorflow model.

    References:
    - http://konukoii.com/blog/2018/02/19/twitter-sentiment-analysis-using-combined-lstm-cnn-models/
    - For text, CNN -> LSTM (or GRU) doesn't seem to work well, but LSTM -> CNN works really well.

    """
    inp = Input(shape=(maxlen,)) 
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(units, return_sequences=True, dropout=dr, recurrent_dropout=dr))(x)
    x = Conv1D(filters=64, kernel_size=2, padding='valid', kernel_initializer="he_uniform")(x)
    x = Dropout(dr)(x)
    # x = MaxPooling1D(pool_size=2)(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                    optimizer=Adam(lr=lr, decay=lr_d),
                    metrics=['accuracy']) 

    return model

In [2]:
model = get_model(maxlen, nb_words, embed_size,
                          embedding_matrix, lr=1e-3, lr_d=0,
                          units=128, dr=0.5)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 150, 300)     28289400    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 150, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 150, 256)     330240      spatial_dropout1d[0][0]          
_______________________________________________________________________________________